In [ ]:
from bs4 import BeautifulSoup
import re

with open(
    "/home/sgmdlt/Work/MZ/es/2025/07/03/RS/03RS0006/УК/03RS0006_2025-07-03_1-268/31-10-2025.16:51:17.html",
    "r",
    errors="replace",
) as f:
    html = f.read()
    html = re.sub(r"</th>\s+<tr>\s+<tr>", "</th></tr><tr>", html, flags=re.IGNORECASE)
    soup = BeautifulSoup(html, "lxml")


from bs4 import BeautifulSoup, Tag
from typing import Union, List, Dict, Any


def _text(el: Tag) -> str:
    return el.get_text(" ", strip=True) if isinstance(el, Tag) else ""


def _has_nested_table(cell: Tag) -> bool:
    return isinstance(cell, Tag) and bool(cell.select(":scope table"))


async def _parse_table(tab: Tag) -> Union[Dict[str, Any], List[Dict[str, Any]]]:
    """
    Специально под структуру:
      - 'ДЕЛО' (2 колонки, ключи в <b>) -> dict
      - 'ДВИЖЕНИЕ ДЕЛА' / 'ЛИЦА' / 'СТОРОНЫ' (много колонок, шапка в <b>) -> list[dict]
    """
    # Список строк: сначала tbody>tr, потом fallback на table>tr
    trs_all = tab.select(":scope > tbody > tr") or tab.select(":scope > tr")
    if not trs_all:
        return {}

    # 1) Отбрасываем чисто-шапочные строки с <th colspan=...>
    data_rows: List[Tag] = []
    for tr in trs_all:
        # если это строка с секционным заголовком, пропускаем
        if tr.select(":scope > th[colspan]"):
            continue
        # пустые технические строки тоже пропустим
        tds = tr.select(":scope > td")
        if not tds:
            continue
        data_rows.append(tr)

    if not data_rows:
        return {}

    # 2) Определяем тип таблицы по первой НЕ-шапочной строке
    first = data_rows[0]
    first_tds = first.select(":scope > td")

    # ---- ВАРИАНТ A: карточка 'ДЕЛО' — строго 2 ячейки в строке
    if len(first_tds) == 2:
        result: Dict[str, Any] = {}
        for tr in data_rows:
            tds = tr.select(":scope > td")
            if len(tds) != 2:
                # встречаются спорадические r/шум — пропустим
                continue
            k_cell, v_cell = tds

            key = _text(k_cell)
            if not key:
                # иногда ключ в <b> — но _text и так его берёт
                continue

            if _has_nested_table(v_cell):
                result[key] = await _parse_table(v_cell.select_one(":scope table"))
            else:
                result[key] = _text(v_cell)
        return result

    # ---- ВАРИАНТ B: многостолбцовые таблицы — заголовки берём по <b>
    # Ищем первую строку, где в ячейках есть <b> (обычно это и есть строка шапки)
    header_tr: Tag = None
    headers: List[str] = []

    # Идём по data_rows и ищем строку, у которой кол-во <b> >= 2
    for tr in data_rows:
        bolds = [b for b in tr.select(":scope > td > b")]
        if len(bolds) >= 2:
            header_tr = tr
            headers = [_text(b) for b in bolds]
            break

    # Если жирных не нашли (редко), берём тексты первой data-строки как заголовки
    if header_tr is None:
        header_tr = first
        headers = [_text(td) for td in first_tds]

    # Собираем строки данных, начиная со строки ПОСЛЕ header_tr
    rows: List[Dict[str, Any]] = []
    header_seen = False
    for tr in data_rows:
        if not header_seen:
            if tr is header_tr:
                header_seen = True
            continue  # пропустить строку-шапку
        tds = tr.select(":scope > td")
        if not tds:
            continue
        row: Dict[str, Any] = {}
        for k, v in zip(headers, tds, strict=False):
            if not k:
                continue
            if _has_nested_table(v):
                row[k] = await _parse_table(v.select_one(":scope table"))
            else:
                row[k] = _text(v)
        if row:
            rows.append(row)

    return rows


# ---------- Совместимый адаптер (имя и сигнатура как у вас) ----------


async def tab_to_dict(tab: Tag) -> Union[Dict[str, Any], List[Dict[str, Any]]]:
    return await _parse_table(tab)


print(await tab_to_dict(soup))

# ---------- ВСТРОЙКА В ВАШУ ФУНКЦИЮ ----------


async def fill_raw_tables(formatted_case: Dict[str, Any], soup: BeautifulSoup) -> None:
    """
    Встраивает парсер таблиц в ваш цикл построения formatted_case["raw_tables"].
    Мутирует formatted_case по месту.
    """
    formatted_case["raw_tables"] = {}

    # Проходим каждую вкладку
    tables = soup.find_all("table")

    # Если таблица одна
    if len(tables) == 1:
        formatted_case["raw_tables"][tables.th.get_text()] = await tab_to_dict(
            tables[0]
        )

    # Если таблиц несколько
    else:
        # Имя для каждой таблицы берём из первой строки, где есть [colspan]
        #
        for table in tables:
            table_name = table.th.get_text()
            content = await tab_to_dict(table)
            formatted_case["raw_tables"][table_name] = content


formated_case = {}
await fill_raw_tables(formated_case, soup)
print(formated_case)

{}


KeyError: 'ДЕЛО'

In [34]:
from bs4 import BeautifulSoup
import re

with open(
    "/home/sgmdlt/Work/MZ/es/2025/07/03/RS/03RS0006/УК/03RS0006_2025-07-03_1-268/31-10-2025.16:51:17.html",
    "r",
    errors="replace",
) as f:
    html = f.read()
    html = re.sub(r"</th>\s+<tr>\s+<tr>", "</th></tr><tr>", html, flags=re.IGNORECASE)
    soup = BeautifulSoup(html, "lxml")

for table in soup.find_all("table"):
    print(table.th.get_text())
    print(await _parse_table(table))

ДЕЛО
{'Уникальный идентификатор дела': '03RS0006-01-2025-004154-95', 'Дата поступления': '03.07.2025', 'Судья': 'Салимгареев И.Р.', 'Дата рассмотрения': '30.07.2025', 'Результат рассмотрения': 'Вынесен ПРИГОВОР', 'Признак рассмотрения дела': 'Рассмотрено единолично судьей'}
ДВИЖЕНИЕ ДЕЛА
[{'Наименование события': 'Регистрация поступившего в суд дела', 'Дата': '03.07.2025', 'Время': '14:40', 'Место проведения': '', 'Результат события': '', 'Основание для выбранного результата события': '', 'Примечание': '', 'Дата размещения': '03.07.2025'}, {'Наименование события': 'Передача материалов дела судье', 'Дата': '03.07.2025', 'Время': '15:03', 'Место проведения': '', 'Результат события': '', 'Основание для выбранного результата события': '', 'Примечание': '', 'Дата размещения': '03.07.2025'}, {'Наименование события': 'Решение в отношении поступившего уголовного дела', 'Дата': '15.07.2025', 'Время': '16:25', 'Место проведения': '', 'Результат события': 'Назначено судебное заседание', 'Основани